# RAGを用いた業務改善シナリオ

## シナリオの概要
ある打合せで決まったネクストアクションに関して、情報収集とレポーティングをサポートする

- Step1- ①動画の内容（今回は記者会見）からネクストアクションの抽出を行う

- Step2- ②動画からネクストアクションの抽出を行う

- Step3- ③ネクストアクションに関連する社内・社外情報を収集する。

- Step4④ネクストアクションに応じて情報をまとめる。

- Step5⑤各種フォーマット（ppt, word等）で資料を作成する。

# 事前準備：必要なソフトウェアをインストール

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install python-pptx python-docx
!pip install unstructured
!pip install faiss-gpu
!pip install requests
!pip install moviepy
!pip install --upgrade moviepy
!pip install pypdf
!pip install getpass

# Step1- ①動画の内容（今回は記者会見）からネクストアクションの抽出を行う
## 動画データ（YouTube）をテキストデータに変換

In [ ]:
from moviepy.editor import VideoFileClip
import requests
import json
import os
import time

def process_video_trim_upload(input_video_path, API_KEY, output_audio_path="/content/video.mp3", max_duration=1000):
    """
    動画から音声を抽出し、Gladia APIを使用して音声をアップロードおよび転写します。

    :param input_video_path: 入力動画ファイルのパス
    :param API_KEY: Gladia APIのAPIキー
    :param output_audio_path: 出力音声ファイルのパス（デフォルトは "/content/video.mp3"）
    :param max_duration: 最大音声時間（秒）。デフォルトは600秒（10分）
    :param polling_interval: ポーリングの間隔（秒）。デフォルトは10秒
    :param max_attempts: 最大ポーリング試行回数。デフォルトは30回（約5分）
    :return: 転写されたデータ（辞書形式）またはエラーメッセージ
    """
    try:
        # 1. 動画から音声を抽出
        print("動画から音声を抽出中...")
        video = VideoFileClip(input_video_path)
        audio = video.audio

        if audio is None:
            return {"error": "この動画には音声が含まれていません。"}

        duration = audio.duration
        print(f"音声の総時間: {duration} 秒")

        if duration > max_duration:
            print(f"音声が{max_duration/60}分を超えているため、最初の{max_duration/60}分のみを保存します。")
            audio = audio.subclip(0, max_duration)

        # 音声を一時ファイルに書き出す
        audio.write_audiofile(output_audio_path)
        print(f"音声が正常に保存されました: {output_audio_path}")

        # 2. 音声ファイルをGladia APIにアップロード
        print("音声ファイルをアップロード中...")
        upload_url = "https://api.gladia.io/v2/upload"
        files = {'audio': (output_audio_file, open(output_audio_file, 'rb'), 'audio/mpeg')}
        headers = {
            "x-gladia-key": API_KEY
        }
        upload_response = requests.post(upload_url, files=files, headers=headers)

        if upload_response.status_code != 200:
            return {"error": f"アップロードに失敗しました: {upload_response.text}"}

        upload_json = upload_response.json()
        audio_url = upload_json.get("audio_url")
        if not audio_url:
            return {"error": "アップロードされた音声のURLが取得できませんでした。"}

        print(f"音声がアップロードされました: {audio_url}")
        return audio_url

    except Exception as e:
        return {"error": f"エラーが発生しました: {str(e)}"}

def request_transcription(audio_url, API_KEY):
    # 3. 転写リクエストを送信
    print("書き起こしリクエストを送信中...")
    transcription_url = "https://api.gladia.io/v2/transcription"
    transcription_data = {
      "audio_url": audio_url,
        "diarization": True,
        "diarization_config": {
            "number_of_speakers": 2,
            "min_speakers": 1,
            "max_speakers": 5
        },
        # 必要に応じて以下のオプションを有効化できます
        # "translation": True,
        # "translation_config": {
        #     "model": "base",
        #     "target_languages": ["fr", "en"]
        # },
        # "subtitles": True,
        # "subtitles_config": {
        #     "formats": ["srt", "vtt"]
        # },
        "detect_language": True,
        "enable_code_switching": False
    }

    headers = {
        'Content-Type': 'application/json',
        'x-gladia-key': API_KEY,
    }

    transcription_response = requests.post(transcription_url, headers=headers, data=json.dumps(transcription_data))

    transcription_json = transcription_response.json()
    print("書き起こしリクエスト完了しました。ID:"+ transcription_json["id"])
    return transcription_json


def get_transcription_result(transcribed_data, api_key, poll_interval=10, max_retries=60):
    """
    トランスクリプションの結果が 'done' になるまでポーリングします。

    :param transcribed_data: トランスクリプション要求のレスポンスデータ（辞書型）
    :param api_key: Gladia APIキー
    :param poll_interval: ポーリング間隔（秒）
    :param max_retries: 最大リトライ回数
    :return: トランスクリプション結果のデータ（辞書型）またはNone
    """
    result_id = transcribed_data["id"]
    if not result_id:
        print("エラー: 'id' が transcribed_data に含まれていません。")
        return None

    url = f"https://api.gladia.io/v2/transcription/{result_id}"
    headers = {"x-gladia-key": api_key}

    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f"エラー: APIリクエストが失敗しました。ステータスコード: {response.status_code}")
                print(f"レスポンス内容: {response.text}")
                return None

            result_data = response.json()
            status = result_data.get('status')

            print(f"現在のステータス: {status}")

            if status == 'done':
                print("トランスクリプションが完了しました。")
                return result_data
            elif status in ['failed', 'canceled']:
                print(f"トランスクリプションが '{status}' 状態で終了しました。")
                return None
            else:
                print(f"トランスクリプションはまだ完了していません。{poll_interval}秒後に再試行します。")

        except requests.exceptions.RequestException as e:
            print(f"リクエスト中にエラーが発生しました: {e}")
            return None
        except json.JSONDecodeError:
            print("レスポンスのJSON解析中にエラーが発生しました。")
            return None

        retries += 1
        time.sleep(poll_interval)

    print("最大リトライ回数に達しました。トランスクリプションが完了していません。")
    return None

def process_transcription(result_data, output_file_path):
    """
    トランスクリプションデータを処理し、スピーカーごとにテキストファイルに出力します。

    :param result_data: トランスクリプション結果のデータ（辞書型）
    :param output_file_path: 出力するテキストファイルのパス
    """
    # 現在のスピーカーのIDを保持する変数を初期化
    current_speaker = None
    # 現在のスピーカーのトランスクリプトを保持する変数を初期化
    current_transcript = ""

    # 出力ファイルを開く（上書きモード）
    with open(output_file_path, 'w', encoding='utf-8') as f:
        # utterancesをループして各発話を処理
        for u in result_data["result"]["transcription"]["utterances"]:
            # 現在のスピーカーが前のスピーカーと同じ場合
            if u["speaker"] == current_speaker:
                # トランスクリプトを結合
                current_transcript += " " + u["text"]
            else:
                # 現在のスピーカーが前のスピーカーと異なる場合（または最初の発話の場合）
                if current_speaker is not None:
                    # 前のスピーカーのトランスクリプトをファイルに書き込む
                    f.write(f"[Speaker: {current_speaker}] {current_transcript}\n")

                # 新しいスピーカーのIDとトランスクリプトを更新
                current_speaker = u["speaker"]
                current_transcript = u["text"]

        # 最後のスピーカーのトランスクリプトをファイルに書き込む
        if current_transcript:
            f.write(f"[Speaker: {current_speaker}] {current_transcript}\n")
            print(f"[Speaker: {current_speaker}] {current_transcript}")

    print(f"トランスクリプション結果が '{output_file_path}' に保存されました。")

In [ ]:
import os
import openai
from getpass import getpass
# APIキーを安全に入力
api_key = getpass("OpenAIのAPIキーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = os.getenv("OPENAI_API_KEY")

api_key = getpass("GRADIAのAPIキーを入力してください: ")
os.environ["GR_API_KEY"] = api_key
GR_API_KEY = os.getenv("GR_API_KEY")

In [ ]:
#ここで事前お配りしたファイルを左のフォルダにアップロードしていただけると助かります！


#動画ファイルのパスは左のファイル部分でお願いします
audio_file= "/content/testmovie.mp4"
# audio_file= "/content/2024ビジネスアップデート.mp4"
output_audio_file = "/content/video.mp3"
OUTPUT_FILE_PATH="/content/transcribed_text.txt"

audio_url = process_video_trim_upload(audio_file, GR_API_KEY)
transcription_response = request_transcription(audio_url, GR_API_KEY)
result_data = get_transcription_result(transcription_response, GR_API_KEY)z
process_transcription(result_data, OUTPUT_FILE_PATH)


# Step2- ②動画からネクストアクションの抽出を行う
## 書き起こしデータからネクストアクションの抽出

In [ ]:
import os
import re
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# OpenAI APIキーの設定
# os.environ["OPENAI_API_KEY"] = "sk-KlvCw7CI3UmHnyztvFslkMT0E8pLhfnA4HXB6uBiD8T3BlbkFJNI2-wy1yA1PkA_xuM5HSS1-ltrwvJaupbKvtE51fwA"
model_name='gpt-4o-mini'

def initialize_llm(temperature=0.0, model_name="gpt-3.5-turbo"):
    """
    ChatOpenAIモデルを初期化します。
    """
    # OpenAI APIキーの設定（環境変数を使用）
    api_key = os.environ.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 'OPENAI_API_KEY' を設定してください。")
    os.environ["OPENAI_API_KEY"] = api_key

    # ChatOpenAIクライアントを初期化
    llm = ChatOpenAI(temperature=temperature, model_name=model_name)
    return llm

def read_transcription(file_path):
    """
    指定されたパスから書き起こしテキストを読み込みます。
    """
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"ファイルが見つかりません: {file_path}")
    with open(file_path, 'r', encoding='utf-8') as file:
        transcription_text = file.read()
    return transcription_text

def split_text(text, chunk_size=5000, chunk_overlap=1000):
    """
    テキストを指定されたチャンクサイズとオーバーラップで分割します。
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    text_chunks = text_splitter.split_text(text)
    return text_chunks

def summarize_chunks(llm, text_chunks):
    """
    各チャンクを要約します。
    """
    summaries = []
    prompt_template = """
        以下のテキストの要点をまとめてください。
        テキスト:
        \"\"\"
        {text}
        \"\"\"
        要約:
        """
    prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
    chain = LLMChain(llm=llm, prompt=prompt)
    for chunk in text_chunks:
        summary = chain.run(chunk)
        summaries.append(summary.strip())
    return summaries

def generate_final_actions(llm, combined_summary):
    """
    結合された要約から5つの具体的なアクションを生成します。
    """
    final_prompt_template = """
      あなたは優秀なビジネスアナリストです。
      以下の要約を読み、企画部がEVに関する市場動向を立てるために必要な5つの具体的なアクションをリスト形式で提案してください。

      要約:
      \"\"\"
      {summary}
      \"\"\"

      出力形式:
      1. アクション1
      2. アクション2
      3. アクション3
      4. アクション4
      5. アクション5
      """
    prompt = PromptTemplate(template=final_prompt_template, input_variables=["summary"])
    final_chain = LLMChain(llm=llm, prompt=prompt)
    actions_text = final_chain.run(combined_summary)
    return actions_text

def parse_actions(actions_text):
    """
    LLMからの出力をパースしてアクションのリストを取得します。
    """
    pattern = r'\d+\.\s*(.*)'
    actions_list = re.findall(pattern, actions_text)
    return actions_list


In [ ]:
# LLMを初期化
llm = initialize_llm()

# 書き起こしテキストを読み込む
transcription_file_path = "/content/transcribed_text.txt"
#意図していない場所に生成された場合
# transcription_file_path = "./content/transcribed_text.txt"

transcription_text = read_transcription(transcription_file_path)


In [ ]:
# テキストを分割
text_chunks = split_text(transcription_text)

# 各チャンクを要約
summaries = summarize_chunks(llm, text_chunks)

# 要約を結合
combined_summary = "\n".join(summaries)

# 最終的なアクションを生成
actions_text = generate_final_actions(llm, combined_summary)

# アクションをリスト形式にパース
actions_list = parse_actions(actions_text)

# 結果を表示
print("企画部が取るべき5つのアクション:")
for idx, action in enumerate(actions_list, 1):
    print(f"{idx}. {action}")

In [ ]:
import pandas as pd

# actions_listをDataFrameに変換し、データフレームとして表示
df = pd.DataFrame({'actions': actions_list})

In [ ]:
# データフレームの中を表示
# これで議事録の録画ファイルからネクストアクションを取り出しました（今回は５つにしています
df

# Step2: RAG用の検索クエリの作成
さきほど動画からネクストアクションまで落とすことができたため、ここからアクションごとにどのようなクエリーを出せばよいかもChatGPTに作成してもらいます

In [ ]:
# タスクから質問を作る関数
def create_RAG_query(text_action):
    llm = ChatOpenAI(temperature=0.0, model=model_name)
    prompt = PromptTemplate(
        input_variables=["text_action"],
        template="""
        以下の情報収集タスクを実行するために、chatGPTを使ったRAGシステムに問い合わせるための質問を１つだけ作成して下さい。その際はRAGのクエリーとして最適な文章とし精度が高くなるようなクエリーにしてください。情報が不足している場合、情報を補ってよりよいクエリーを生成してください。
        ### 情報収集タスク:
        　{text_action}
        """
    )
    #★★このプロンプトはより高度で精度が高くなるプロンプトにすることが重要ですので、適宜上記のプロンプトを変更してください。
    chain = LLMChain(llm=llm, prompt=prompt)
    query = chain.run(text_action=text_action)
    return query

In [ ]:
# pandasの各行について、create_RAG_queryを実行し、rag_queryという列名で追加
df['rag_query'] = df['actions'].apply(create_RAG_query)

In [ ]:
# RAGに問い合わせるクエリーがきちんと作成できているか、結果を確認する
df

# Step3:情報の検索とまとめ：RAGの実行
さきほど作成したクエリーをもとにRAGシステムにクエリーを投げていきます。
今回はPDFを対象としていますが、本来は複数ファイルやフォルダごとやストレージ（onedrive, Sharepoint)などでも対応できます

In [ ]:
import os
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool
from langchain.document_loaders import PyPDFLoader

In [ ]:
# PDFのデータベース化（こちらも事前にお配りしたtoyota.pdfを使います
# 今回は混乱を避けるために１ファイルにしますが、複数ファイルでも問題ありません
loader = PyPDFLoader("/content/toyota.pdf")

#へんなところにファイルがある場合
# loader = PyPDFLoader("./content/toyota.pdf")

# loader = PyPDFLoader("/content/toyota_2024_4q_presentation_jp.pdf")

In [ ]:
pages = loader.load_and_split()
# ページ指定を変えると中身がわかります（例：1 -> 2
pages[1]

In [ ]:
# この情報をベクトル化していき、キーワードの合致ではなく、意味の近さで検索するようにします
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(pages, embeddings)

In [ ]:
# ベクトルの次元数を取得（インデックスから取得）
vector_dimension = vectorstore.index.d
print(f"ベクトルの次元数: {vector_dimension}")

In [ ]:
# ベクトル情報の確認
vectors = vectorstore.index.reconstruct_n(0, vectorstore.index.ntotal)

# 最初のいくつかのベクトルを表示
for i, vector in enumerate(vectors[:5]):
    print(f"Vector {i}: {vector}")

In [ ]:
# RAGシステムを使用して関連情報の参照先と回答結果を取得する関数
def search_related_information(query, documents):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents, embeddings)
    # リトリーバーの生成
    retriever = vectorstore.as_retriever()
    # ベクトルストアから情報を取得
    qa = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model=model_name),
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    result = qa({"query": query})
    # 参照先のドキュメント情報を取得
    documents = result['source_documents']
    # ドキュメントから関連度の最も高いページの内容を取得
    page_content = documents[0].page_content
    # 参照先の情報を表示
    print("source_file:",documents[0].metadata['source'])
    print("source_content:",page_content)
    # 回答結果を取得
    answer = result['result']
    return page_content

def create_RAG_answer(rag_query,reference_text):
    llm = ChatOpenAI(temperature=0.0, model=model_name)
    prompt = PromptTemplate(
        input_variables=["rag_query","reference_text"],
        template="""
        下記の質問について、参照情報を参考して回答して下さい。
        ### 質問
          {rag_query}
        ### 参照情報:
        　{reference_text}
        """
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    answer = chain.run(rag_query=rag_query, reference_text=reference_text)
    return answer


In [ ]:
# 先ほど作成したRAG用のクエリを実行し、RAGの参照先のデータを取得
# pandasの各行について、create_RAG_queryを実行し、rag_queryという列名でDataFrameに新しい列を追加
df['reference_text'] = df['rag_query'].apply(
    lambda x: pd.Series(search_related_information(x, pages))
)

In [ ]:
# RAGの参照先のデータを利用し、回答を生成
# rag_answerという列名でDataFrameに新しい列を追加
df['rag_answer'] = df.apply(lambda row: create_RAG_answer(row['rag_query'], row['reference_text']), axis=1)

In [ ]:
#実際のデータの中身がどうなったか確認します
df

In [ ]:
# この時点でCSVにも出力しておきます
df.to_csv("rag.csv")

# Step4: 適切なフォーマットに変換していく（例：パワーポイントの作成
これまでの工程では、録画ファイルからネクストアクションを抽出し、ネクストアクションからRAGで利用するクエリーを作成。
そこから実際にファイルやフォルダにアクセスし、「参照する文章」「RAGとして問い合わせた回答」を取得しました。
実際の業務効率化を考えた場合、pptやwordやメールを使う機会が多いと思いますので、そのフォーマットに変換することを考えます

In [ ]:
import pandas as pd

# 日本語の説明: rag.csvというCSVファイルを読み込み、必要なデータ部分だけdfの中に入れます。
df = pd.read_csv('rag.csv', usecols=['actions', 'rag_query', 'reference_text', 'rag_answer'])

df.head(5)



In [ ]:
# 上でいれているので、飛ばしてOKです！
#今回はAPIキーを直接入力するために、このようなおまじないをいれます。これを走らせると、キーの入力が求めれます。

import os
import openai
from getpass import getpass

# APIキーを安全に入力
api_key = getpass("OpenAIのAPIキーを入力してください: ")
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = os.getenv("OPENAI_API_KEY")




In [ ]:
# 日本語の説明: dfに対して、formatというカラムを追加し、"ppt", "word", "mail"を行の数だけ入力します。
formats = []

# 各行に対してフォーマットを入力
for i in range(len(df)):
    format_input = input(f"行 {i+1} のフォーマットを入力してください (ppt, word, mail): ")
    while format_input not in ["ppt", "word", "mail"]:
        print("無効な入力です。'ppt', 'word', 'mail'のいずれかを入力してください。")
        format_input = input(f"行 {i+1} のフォーマットを入力してください (ppt, word, mail): ")
    formats.append(format_input)

df['format'] = formats



In [ ]:
# 日本語の説明: target_deptとkey_manを定義し、インプット関数で今回のターゲットを定義します。

# ターゲット部門とキーマンをユーザーに入力してもらう
target_dept = input("ターゲット部門を入力してください: ")
key_man = input("キーマンや役職を入力してください: ")

# 入力された値を表示
print(f"ターゲット部門: {target_dept}")
print(f"キーマン: {key_man}")



In [ ]:
from openai import OpenAI
# chatGPTに質問する
# OpenAIクライアントの初期化
client = OpenAI()

# 日本語の説明: ChatGPTでrag_answerのテキストをインプットにし、format_textにGPT4oで変換します。

def format_text(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # GPT-4oのモデル名
        messages=[
            {"role": "system", "content": "あなたは優秀なAIアシスタントです。"},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# promptは可変とし、下でフォーマットごとのプロンプトを作成する

In [ ]:
# 日本語の説明: dfの各行に対して、formatカラムの値に応じてGPT-4oのプロンプトを生成し、format_text関数を使用して変換します。

# 各行に対して処理を行う
for i, row in df.iterrows():
    # rag_answerを取得
    rag_answer = df['rag_answer'].iloc[i]
    print("□□□□□□□□□")
    print("RAGの文章を変更していきます：", rag_answer)

    # formatに応じてプロンプトを生成
    if row['format'] == 'ppt':
        user_prompt = f'''
        {target_dept}の{key_man}様へ提出するプレゼンテーションを作成してください。以下の内容を要点にまとめ、効果的なスライドを作成します。スライドは「スライドタイトル」「キーメッセージ」「ボディ」として構成し、どのような図解を入れるべきかも指示してください。
        # 内容: {rag_answer}
        '''
    elif row['format'] == 'word':
        user_prompt = f'''
        {target_dept}の{key_man}様へ送る報告書を作成してください。以下の内容を4部構成で書いてください。
        導入、分析、提案、結論を含む形式で記述します。マークダウン記法で記載し、PREP法を意識してください。
        内容: {rag_answer}
        '''
    elif row['format'] == 'mail':
        user_prompt = f'''
        {target_dept}の{key_man}様へ送るメールを作成してください。次のステップを提案する内容です。
        件名、挨拶、本文、締めの言葉を含め、丁寧かつ簡潔にまとめてください。
        内容: {rag_answer}
        '''

    # GPT-4oでテキストを変換
    try:
        print(user_prompt)
        formatted_text = format_text(user_prompt)
    except UnicodeEncodeError as e:
        print(f"エラーが発生しました: {e}")
        formatted_text = "エラー: テキストの変換に失敗しました。"

    # 変換されたテキストを新しいカラムに追加
    df.at[i, 'format_text'] = formatted_text
    print("■■■■■■■■■■■")
    print(f"行目: {i}, フォーマット: {row['format']}, 変換されたテキスト:\n{formatted_text}\n")


In [ ]:
# 実際に出力する
df.head(5)

# Step5: 各種フォーマット（ppt, word等）で資料を作成する。
ここからは、これまで作成したものを使わず、pptやワードで資料が作成できることをお見せします。

In [ ]:
input_data = f"""
目的：{df["actions"][0]}

情報：{df["rag_answer"][0]}
"""

In [ ]:
print(input_data)

## Step5: ①各種フォーマット（ppt, word等）で資料を作成する: pptの参考情報

In [ ]:
def generate_presentation(input_str,  output_filename="meeting_agenda_presentation.pptx"):
    from langchain.prompts import PromptTemplate
    from langchain_openai import ChatOpenAI
    from pptx import Presentation
    from pptx.util import Inches, Pt
    import os

    # OpenAI APIキーの設定（環境変数を使用）
    os.environ["OPENAI_API_KEY"] = "sk-KlvCw7CI3UmHnyztvFslkMT0E8pLhfnA4HXB6uBiD8T3BlbkFJNI2-wy1yA1PkA_xuM5HSS1-ltrwvJaupbKvtE51fwA"

    # 入力文字列から action と input_data をパース
    lines = input_str.strip().split('\n')
    action = ''
    input_data = ''
    for line in lines:
        if line.startswith('目的:'):
            action = line.replace('目的:', '').strip()
        elif line.startswith('情報:'):
            input_data = line.replace('情報:', '').strip()
        else:
            input_data += line.strip() + '\n'

    # OpenAIを使ったチャットモデルを初期化
    llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")

    # プロンプトテンプレートを修正
    slide_template = """
    あなたはPowerPointを自動作成するエージェントです。以下の入力情報に基づいて、情報をまとめるたたきを作成してください。
    各スライドには次の3つの内容を加えてください。１．タイトル、２．リード文（タイトルの下に記載、タイトルで伝えたいメッセージ）、３．リード文を保証する情報の3つ（リード文の下に記載）です。
    ただし、最初のスライドには総括となるまとめを記載するようにしてください。

    目的:
    {action}

    入力情報:
    {input_data}

    各スライドは以下の形式で記述してください：

    [タイトル]
    [リード文]
    [内容]

    「スライドタイトル:」や「スライドのリード文:」といった文言は含めないでください。

    スライド間は「---」で区切ってください。

    """

    # LangChainのプロンプトテンプレートを作成
    prompt = PromptTemplate(
        input_variables=["action","input_data"],
        template=slide_template
    )

    # プロンプトとLLMを組み合わせてチェーンを作成
    chain = prompt | llm

    # LLMを使って、スライドの内容を生成
    generated_content = chain.invoke({"action":action, "input_data": input_data})

    # AIMessageオブジェクトからテキストを取得
    content_text = generated_content.content

    # 生成されたスライドの内容をprint文で確認
    print(f"生成されたスライドの内容:\n{content_text}")

    # PowerPointの自動生成
    def create_presentation(content):
        # 新しいプレゼンテーションを作成
        prs = Presentation()

        # 各スライドの情報を生成された内容から取得
        slides_info = content.split("---")

        for slide_info in slides_info:
            if slide_info.strip():  # 空白のみの行を無視
                slide_lines = slide_info.strip().split("\n", 2)
                if len(slide_lines) >= 2:
                    slide_title = slide_lines[0].strip()
                    lead_sentence = slide_lines[1].strip()
                    components = slide_lines[2].strip() if len(slide_lines) > 2 else ""

                    # 不要な文言を削除
                    for prefix in ["スライドタイトル:", "タイトル:"]:
                        if slide_title.startswith(prefix):
                            slide_title = slide_title.replace(prefix, "").strip()
                    for prefix in ["リード文:", "スライドのリード文:"]:
                        if lead_sentence.startswith(prefix):
                            lead_sentence = lead_sentence.replace(prefix, "").strip()

                    # リード文内の行頭のハイフンを削除
                    lead_sentence = re.sub(r'^\s*-\s*', '', lead_sentence, flags=re.MULTILINE)

                    # 新しいスライドを作成
                    slide = prs.slides.add_slide(prs.slide_layouts[1])

                    # タイトルを設定
                    title = slide.shapes.title
                    title.text = slide_title

                    # コンテンツプレースホルダーを取得
                    content_placeholder = slide.placeholders[1]
                    text_frame = content_placeholder.text_frame
                    text_frame.clear()

                    # リード文を追加（大きめのフォントサイズ、箇条書きなし）
                    p_lead = text_frame.paragraphs[0]
                    p_lead.text = lead_sentence
                    p_lead.font.size = Pt(24)  # リード文のフォントサイズを設定
                    p_lead.level = 0  # レベルを0に設定して箇条書きを無効にする

                    # 構成要素を追加（小さめのフォントサイズ、箇条書きあり）
                    if components:
                        # 構成要素を行ごとに分割
                        components = re.sub(r'^\s*-\s*', '', components, flags=re.MULTILINE)
                        component_lines = components.strip().split('\n')
                        for line in component_lines:
                            p_component = text_frame.add_paragraph()
                            p_component.text = line.strip()
                            p_component.font.size = Pt(18)  # 構成要素のフォントサイズを設定
                            p_component.level = 1  # レベルを1に設定して箇条書きを適用

            else:
                continue  # 空のスライド情報をスキップ

        # プレゼンテーションを保存
        prs.save(output_filename)

    # プレゼンテーションを作成
    create_presentation(content_text)

    print("プレゼンテーションが作成されました。ファイル名: {}".format(output_filename))


In [ ]:
generate_presentation(input_data, output_filename="my_presentation.pptx")

## Step5: ②各種フォーマット（ppt, word等）で資料を作成する: wordの参考情報

In [ ]:
import os
import re
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from docx import Document
from docx.shared import Pt, RGBColor  # RGBColorをインポート

def generate_word_document(input_str, output_filename="generated_document.docx"):
    # OpenAI APIキーの設定（環境変数を使用）
    api_key = os.environ.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 'OPENAI_API_KEY' を設定してください。")
    os.environ["OPENAI_API_KEY"] = api_key

    # OpenAIを使ったチャットモデルを初期化
    llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")

    # 入力文字列から action と input_data をパース
    lines = input_str.strip().split('\n')
    action = ''
    input_data = ''
    for line in lines:
        if line.startswith('目的:'):
            action = line.replace('目的:', '').strip()
        elif line.startswith('情報:'):
            input_data = line.replace('情報:', '').strip()
        else:
            input_data += line.strip() + '\n'

    # プロンプトテンプレートを修正
    document_template = """
    あなたはWord文書を自動作成するエージェントです。以下の入力情報に基づいて、情報をまとめた文書を作成してください。
    各セクションに含める内容は、見出しとリード文（見出しの下に記載、見出しで伝えたいメッセージ）、そしてリード文を補足する情報の3つです。
    ただし、最初のセクションには総括となるまとめを記載するようにしてください。

    目的:
    {action}

    入力情報:
    {input_data}

    各セクションは以下の形式で記述してください：

    [見出し]
    [リード文]
    [内容]

    「セクション見出し:」や「リード文:」といった文言は含めないでください。

    セクション間は「---」で区切ってください。

    """

    # LangChainのプロンプトテンプレートを作成
    prompt = PromptTemplate(
        input_variables=["action","input_data"],
        template=document_template
    )

    # プロンプトとLLMを組み合わせてチェーンを作成
    chain = prompt | llm

    # LLMを使って、文書の内容を生成
    generated_content = chain.invoke({"action":action, "input_data": input_data})

    # AIMessageオブジェクトからテキストを取得
    content_text = generated_content.content

    # 生成されたコンテンツをprintで確認
    print("生成されたコンテンツ:\n", content_text)

    # Word文書の作成関数
    def create_document(content):
        # 新しいWord文書を作成
        doc = Document()

        # 各セクションの情報を生成された内容から取得
        sections_info = content.split("---")

        for section_info in sections_info:
            if section_info.strip():  # 空白のみの行を無視
                section_lines = section_info.strip().split("\n", 2)
                if len(section_lines) >= 2:
                    heading = section_lines[0].strip()
                    lead_sentence = section_lines[1].strip()
                    components = section_lines[2].strip() if len(section_lines) > 2 else ""

                    # 不要な文言を削除
                    for prefix in ["セクション見出し:", "見出し:"]:
                        if heading.startswith(prefix):
                            heading = heading.replace(prefix, "").strip()
                    for prefix in ["リード文:", "セクションのリード文:"]:
                        if lead_sentence.startswith(prefix):
                            lead_sentence = lead_sentence.replace(prefix, "").strip()

                    # リード文内の行頭のハイフンを削除
                    lead_sentence = re.sub(r'^\s*-\s*', '', lead_sentence, flags=re.MULTILINE)

                    # 見出しを追加（レベル1）
                    heading1 = doc.add_heading(heading, level=1)
                    # 見出しの文字色を黒に設定
                    for run in heading1.runs:
                        run.font.color.rgb = RGBColor(0, 0, 0)

                    # リード文を追加（レベル2の見出しとして追加）
                    heading2 = doc.add_heading(lead_sentence, level=2)
                    # リード文の文字色を黒に設定
                    for run in heading2.runs:
                        run.font.color.rgb = RGBColor(0, 0, 0)

                    # 構成要素を追加（通常の段落）
                    if components:
                        # 構成要素を行ごとに分割
                        components = re.sub(r'^\s*-\s*', '', components, flags=re.MULTILINE)
                        component_lines = components.strip().split('\n')
                        for line in component_lines:
                            p_component = doc.add_paragraph()
                            run_component = p_component.add_run(line.strip())
                            run_component.font.size = Pt(11)
                            # 構成要素の文字色を黒に設定
                            run_component.font.color.rgb = RGBColor(0, 0, 0)

                else:
                    continue  # 空のセクション情報をスキップ

        # 文書を保存
        doc.save(output_filename)

    # Word文書を作成
    create_document(content_text)

    print("Word文書が作成されました。ファイル名: {}".format(output_filename))


In [ ]:
generate_word_document(input_data, output_filename="generated_document.docx")

## Step5: ③各種フォーマット（ppt, word等）で資料を作成する: mailの参考情報

In [ ]:
import os
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

def generate_email_content(input_str):
    # OpenAI APIキーの設定（環境変数を使用）
    api_key = os.environ.get('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OpenAI APIキーが設定されていません。環境変数 'OPENAI_API_KEY' を設定してください。")
    os.environ["OPENAI_API_KEY"] = api_key

    # OpenAIを使ったチャットモデルを初期化
    llm = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo")

    # 入力文字列から action と input_data をパース
    lines = input_str.strip().split('\n')
    action = ''
    input_data = ''
    for line in lines:
        if line.startswith('目的:'):
            action = line.replace('目的:', '').strip()
        elif line.startswith('情報:'):
            input_data = line.replace('情報:', '').strip()
        else:
            input_data += line.strip() + '\n'

    # メール作成用のプロンプトテンプレート
    email_template = """
    あなたはビジネス文書の作成に長けたプロフェッショナルです。以下の入力情報に基づいて、関連部署に報告するメール文を作成してください。

    **要件:**
    - 件名は適切なものを設定してください。
    - メール本文には以下の点を含めてください:
        - 敬語を使用し、丁寧な表現を心がける
        - 報告内容の概要を簡潔に述べる
        - 添付資料がある場合は、その旨を記載する
        - 必要に応じて今後のアクションや問い合わせ先を明記する

    **入力情報:**
    {input_data}

    **出力形式:**
    ```
    件名：

    [メール本文]
    ```
    """

    # LangChainのプロンプトテンプレートを作成
    prompt = PromptTemplate(
        input_variables=["input_data"],
        template=email_template
    )

    # プロンプトとLLMを組み合わせてチェーンを作成
    chain = prompt | llm

    # LLMを使って、メールの内容を生成
    generated_email = chain.invoke({
        "input_data": input_data
    })

    # AIMessageオブジェクトからテキストを取得
    email_text = generated_email.content

    # 結果を表示
    print("生成されたメールの内容:")
    print(email_text)

    # 必要に応じて、メールの内容をファイルに保存することも可能です
    # with open("email_content.txt", "w", encoding="utf-8") as f:
    #     f.write(email_text)

    return email_text


In [ ]:
# 出来上がったものをフォーマットを作成する
generate_email_content(input_data)

# おまけ：エージェントによる資料作成の自動化

In [ ]:
#資料作成エージェント（先ほど作成したコードの実行）
def agent_function(input_str):
    tools = [
          Tool(
              name="generate_presentation",
              func=generate_presentation,
              description="このツールはインプットを受け取り、含まれたアクションとそのための情報を取得します。それらを元に、まとめ資料をpptで作成します。",
          ),
          Tool(
              name = "generate_word_document",
              func=generate_word_document,
              description="このツールはインプットを受け取り、含まれたアクションとそのための情報を取得します。それらを元に、まとめ資料をwordで作成します。",
          ),
          Tool(
              name = "generate_email_content",
              func=generate_email_content,
              description="このツールはインプットを受け取り、含まれたアクションとそのための情報を取得します。それらを元に、まとめ報告するメール文章を作成します。"
          ),
      ]
    chat_agent = initialize_agent(
        tools,
        llm=ChatOpenAI(temperature=0, model=model_name),
        agent = "zero-shot-react-description",
        verbose=True,
        system_message="あなたは親切なアシスタントです。インプットされた目的と情報を元に",
    )

    # エージェントへの入力を作成
    agent_input = f"各種関係者に目的に応じた情報を共有したいので、ppt,word,メール作成を手伝ってください。:\n目的：{action}\n情報：{rag_result} "
    result = chat_agent.run(agent_input)
    result_ja = translate_to_japanese(result)
    return result_ja

In [ ]:
agent_function(input_str)